In [29]:
import pandas as pd # Data Transformation
import pytest       # Makes it easy to write small, readable tests
import os
from dotenv import load_dotenv
import psycopg2
from psycopg2 import sql
import sqlalchemy
from sqlalchemy import create_engine, text, exc

In [30]:
# Test query to check connection
test_query = "SELECT version();"

try:
    # Execute the test query
    with db_engine.connect() as connection:
        result = connection.execute(text(test_query))
        version = result.fetchone()
    print("Connected to PostgreSQL successfully. Version:", version[0])
except Exception as e:
    print(f"Error connecting to PostgreSQL: {e}")

2024-06-14 17:39:09,782 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-14 17:39:09,784 INFO sqlalchemy.engine.Engine SELECT version();
2024-06-14 17:39:09,786 INFO sqlalchemy.engine.Engine [generated in 0.00402s] {}
2024-06-14 17:39:09,790 INFO sqlalchemy.engine.Engine ROLLBACK
Connected to PostgreSQL successfully. Version: PostgreSQL 16.3 (Debian 16.3-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit


In [31]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text, exc

# Load environment variables from .env file
load_dotenv()

# Retrieve individual components from environment variables
user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD')
host = os.getenv('POSTGRES_HOST')
db_name = os.getenv('POSTGRES_DB')
port = os.getenv('POSTGRES_PORT', 5432)  # Default to 5432 if not set

# Ensure the connection URI is retrieved successfully
if not all([user, password, host, db_name]):
    raise ValueError("One or more environment variables for the database connection are not set")

# Construct the connection URI
connection_uri = f"postgresql://{user}:{password}@{host}:{port}/{db_name}"

# Ensure the connection URI is retrieved successfully
if connection_uri is None:
    raise ValueError("DATABASE_URL environment variable is not set")

# Create the database engine
db_engine = create_engine(connection_uri, echo=True)  # Set echo=True for SQL logging

# SQL query to create schemas
create_schemas_sql = """
CREATE SCHEMA IF NOT EXISTS churn_bronze;
CREATE SCHEMA IF NOT EXISTS churn_silver;
CREATE SCHEMA IF NOT EXISTS churn_gold;
"""

try:
    # Execute the SQL query to create schemas
    with db_engine.connect() as connection:
        print("Attempting to create schemas...")
        connection.execute(text(create_schemas_sql))
    print("Schemas created successfully.")
except exc.SQLAlchemyError as e:
    print(f"Error creating schemas: {e}")

# SQL query to check the created schemas
check_schemas_sql = """
SELECT schema_name
FROM information_schema.schemata
WHERE schema_name IN ('churn_bronze', 'churn_silver', 'churn_gold');
"""

try:
    # Execute the SQL query to check schemas and fetch the result
    with db_engine.connect() as connection:
        print("Checking for the existence of schemas...")
        result = connection.execute(text(check_schemas_sql))
        schemas = result.fetchall()
    if schemas:
        print("Schemas present in the database:")
        for schema in schemas:
            print(schema[0])
    else:
        print("No schemas found.")
except exc.SQLAlchemyError as e:
    print(f"Error checking schemas: {e}")

# Close the engine connection
db_engine.dispose()


2024-06-14 17:39:09,848 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-06-14 17:39:09,850 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-06-14 17:39:09,853 INFO sqlalchemy.engine.Engine select current_schema()
2024-06-14 17:39:09,855 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-06-14 17:39:09,859 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-06-14 17:39:09,861 INFO sqlalchemy.engine.Engine [raw sql] {}
Attempting to create schemas...
2024-06-14 17:39:09,864 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-14 17:39:09,866 INFO sqlalchemy.engine.Engine 
CREATE SCHEMA IF NOT EXISTS churn_bronze;
CREATE SCHEMA IF NOT EXISTS churn_silver;
CREATE SCHEMA IF NOT EXISTS churn_gold;

2024-06-14 17:39:09,868 INFO sqlalchemy.engine.Engine [generated in 0.00405s] {}
2024-06-14 17:39:09,872 INFO sqlalchemy.engine.Engine ROLLBACK
Schemas created successfully.
Checking for the existence of schemas...
2024-06-14 17:39:09,876 INFO sqlalchemy.engine.Engine 